In [2]:
import pandas as pd

df = pd.read_excel("dataset/train/Data Latih BDC.xlsx")
df = df.drop(['ID','tanggal','nama file gambar'],axis=1)

In [3]:
df

,label,judul,narasi
0,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
1,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
2,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
3,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
4,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .
...,...,...,...
4226,1,: KPK Dilarang Membawa Brimob Bersenjata Masuk...,KPK Dilarang Membawa Brimob Bersenjata Masuk G...
4227,1,Foto pejabat keuangan dibawah palu arit,Jangan mau dialihkan kepada pakaian adat… FOKU...
4228,1,Gambar Denny Siregar Musuh Warga Tasikmalaya d...,Akhir nya bisa terkenal bang denny
4229,1,Kaesang: Bapak Saya dengan Kesederhaan Bisa Ni...,Bapak Saya dengan Kesederhaan Bisa Nipu Rakyat...


In [4]:
#input variabel [judul, narasi]
X = df.iloc[:,1:3].values

#Target Variabel [label]
y = df.iloc[:,0].values

print(X.shape)
print(y.shape)

(4231, 2)
(4231,)


In [5]:
import string
from gensim.parsing.preprocessing import remove_stopwords
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

#remove stopword
factory = StopWordRemoverFactory()
stopwords_indo = factory.create_stop_word_remover()
rows, cols = X.shape
for i in range(0, cols):
  for j in range(0, rows):
    review = re.sub('[^a-zA-Z]',' ',X[j,i])
    review = review.lower()
    review = review.translate(str.maketrans('','',string.punctuation)).lower()
    review = stopwords_indo.remove(review)

    review = remove_stopwords(review)
    X[j,i] = review

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# number of times a word appears in the document but is counterbalanced by the number of documents in which it is present.

transform = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
tf_judul = transform.fit_transform(X[:,0]).todense()
tf_narasi = transform.fit_transform(X[:,1]).todense()

In [7]:
import numpy as np
X_mat = np.hstack((tf_judul, tf_narasi))
X_mat.shape

(4231, 10000)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.2, random_state=0)
X_test.shape

(847, 10000)

In [9]:
from sklearn.tree import DecisionTreeClassifier
import pickle
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

# filename = 'model/model.sav'
# pickle.dump(model, open(filename, 'wb'))

e:\Porto Data\hoax-detection\env\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
e:\Porto Data\hoax-detection\env\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [10]:
from sklearn.metrics import f1_score

#calculate performance metrics
F1 = f1_score(y_pred=y_predict, y_true=y_test)
print(F1)

0.8720514653323803


In [11]:
test_predict = model.predict(X_test[10])

e:\Porto Data\hoax-detection\env\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [12]:
print(test_predict[0])

[[0. 0. 0. ... 0. 0. 0.]]
1
